In [110]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torch.utils.data as data

In [111]:
train_data_path = "./train"
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
    std=[0.229, 0.224, 0.225])
])

train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)

In [112]:
transform

Compose(
    Resize(size=(64, 64), interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [113]:
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transform)

test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform)

In [114]:
batch_size = 64
train_data_loader = data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = data.DataLoader(test_data, batch_size=batch_size)

In [115]:
image, label = next(iter(train_data_loader))

In [116]:
image.shape, label.shape

(torch.Size([64, 3, 64, 64]), torch.Size([64]))

In [117]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
        

In [118]:
simplenet = SimpleNet()

In [119]:
simplenet

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [120]:
import torch.optim as optim
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

In [121]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# model.to(device)

In [122]:
device

device(type='cpu')

In [123]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    print(device)
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        for batch in train_loader:
            optimizer.zero_grad()
            input, target = batch
            input = input.to(device)
            target = target.to(device)

            output = model(input)

            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()

            training_loss += loss.data.item()
        training_loss /= len(train_loader)

        model.eval()
        num_correct = 0
        num_examples = 0

        for batch in val_loader:
            input, target = batch
            input = input.to(device)
            output = model(input)

            target = target.to(device)

            loss = loss_fn(output, target)
            valid_loss += loss.data.item()

            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1],
							   target).view(-1)

            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]

        valid_loss /= len(val_loader)

        print("Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}".format(epoch, training_loss,
        valid_loss, num_correct / num_examples))



In [124]:
device

device(type='cpu')

In [125]:
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(),train_data_loader, val_data_loader, 20, device)

cpu
Epoch: 0, Training Loss: 3.22, Validation Loss: 2.61, accuracy = 0.41
Epoch: 1, Training Loss: 1.97, Validation Loss: 1.58, accuracy = 0.67
Epoch: 2, Training Loss: 1.24, Validation Loss: 0.84, accuracy = 0.61
Epoch: 3, Training Loss: 0.60, Validation Loss: 0.65, accuracy = 0.73
Epoch: 4, Training Loss: 0.52, Validation Loss: 0.62, accuracy = 0.68
Epoch: 5, Training Loss: 0.39, Validation Loss: 0.66, accuracy = 0.70
Epoch: 6, Training Loss: 0.36, Validation Loss: 0.63, accuracy = 0.72
Epoch: 7, Training Loss: 0.32, Validation Loss: 0.63, accuracy = 0.72
Epoch: 8, Training Loss: 0.29, Validation Loss: 0.63, accuracy = 0.72
Epoch: 9, Training Loss: 0.26, Validation Loss: 0.66, accuracy = 0.72
Epoch: 10, Training Loss: 0.24, Validation Loss: 0.65, accuracy = 0.71
Epoch: 11, Training Loss: 0.22, Validation Loss: 0.67, accuracy = 0.71
Epoch: 12, Training Loss: 0.20, Validation Loss: 0.68, accuracy = 0.71
Epoch: 13, Training Loss: 0.18, Validation Loss: 0.70, accuracy = 0.72
Epoch: 14, T

In [128]:
torch.save(simplenet, "simplenet")

In [140]:
img = test_data[0][0]

In [149]:
img.numpy().shape

(3, 64, 64)

In [148]:
img = Image.fromarray(img.numpy())

TypeError: Cannot handle this data type: (1, 1, 64), <f4

In [154]:
classes = train_data_loader.dataset.classes

In [155]:
classes

['cat', 'fish']

In [160]:
cat = "/Users/christian_acuna/workspace/beginners-pytorch-deep-learning/chapter2/test/cat/3156111_a9dba42579.jpg"

In [169]:
fish = "/Users/christian_acuna/workspace/beginners-pytorch-deep-learning/chapter2/test/fish/35099721_fbc694fa1b.jpg"

In [170]:
from PIL import Image

def inference(filename, classes, model):
    labels = ['cat', 'fish']
    img = Image.open(filename)
    img = transform(img)
    img = img.unsqueeze(0)

    prediction = model(img)
    prediction = prediction.argmax()
    print(classes[prediction])

In [171]:
inference(cat, classes, simplenet)

cat


In [172]:
inference(fish, classes, simplenet)

fish


In [173]:
net = torch.load("simplenet")

In [174]:
net

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)